In [6]:
import requests
import json
import re
from datetime import datetime
import os
from pathlib import Path
from typing import List, Dict, Any

# 定义数据类型
Word = Dict[str, Any]

def get_hot_search_info() -> List[Word]:
    """
    获取微博热搜数据
    """
    url = 'https://s.weibo.com/top/summary'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Cookie': 'SUB=_2AkMVdRtlf8NxqwJRmfoWy2_lb4V0yQvEieKjKeq-JRMxHRl-yT8XqmYatRB6PvU1ijEk4CykabQQvFhJAy31x99v4Ejs;'
    }

    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'

        if not response.ok:
            print(f"请求失败: {response.status_code} - {response.reason}")
            return []

        # 使用正则表达式匹配热搜数据:cite[1]
        regexp = r'<a href="(\/weibo\?q=[^"]+)".*?>(.+)<\/a>'
        matches = re.findall(regexp, response.text)

        words: List[Word] = []
        for match in matches:
            words.append({
                'url': match[0],
                'title': match[1]
            })

        return words

    except Exception as e:
        print(f"获取微博热搜时出错: {e}")
        return []



def save_data(data: List[Word], file_path: str) -> None:
    """
    保存数据到JSON文件
    """
    try:
        # 确保目录存在
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"数据已保存到: {file_path}")
    except Exception as e:
        print(f"保存数据时出错: {e}")

def main():
    """
    主函数
    """
    # 获取当前日期
    yyyyMMdd = datetime.now().strftime("%Y-%m-%d")

    # 获取热搜数据
    words = get_weibo_hot_search()

    if not words:
        print("未获取到热搜数据")
        return

    # 添加真实URL
    for word in words:
        word['realurl'] = f"https://s.weibo.com/{word['url']}"

    # 文件路径
    raw_dir = Path("raw/weibo-search")
    raw_dir.mkdir(parents=True, exist_ok=True)
    full_path = raw_dir / f"{yyyyMMdd}.json"

    # 检查是否已有数据文件
    words_already_downloaded: List[Word] = []
    if full_path.exists():
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                words_already_downloaded = json.load(f)
        except Exception as e:
            print(f"读取现有数据时出错: {e}")

    # 合并数据
    words_all = merge_words(words, words_already_downloaded)

    # 保存数据
    save_data(words_all, str(full_path))

    print(f"成功获取并保存 {len(words)} 条微博热搜数据")

    # 打印前10条热搜
    print("\n=== 微博热搜TOP10 ===")
    for i, word in enumerate(words[:10], 1):
        print(f"{i}. {word['title']}")

    return words_all

In [7]:
words_all = main()

数据已保存到: raw\weibo-search\2025-09-13.json
成功获取并保存 51 条微博热搜数据

=== 微博热搜TOP10 ===
1. 家国今日如你所愿
2. 教资作文
3. 教资
4. 烈士遗骸安葬仪式
5. 老乡鸡微博置顶回应了预制菜
6. 法考
7. 李昀锐 早知道不接电话了
8. 九三阅兵式上唯一一面空军战旗
9. 黄子韬徐艺洋再跳冰激凌从同事变夫妻
10. 教资作文 梦到哪句写哪句


In [9]:
len(words_all)

57

In [13]:
from typing import List, Dict, Any, TypedDict
# 定义数据类型
class SearchWord(TypedDict):
    query: str
    display_query: str
    url: str

class TopSearch(TypedDict):
    top_search: Dict[str, List[SearchWord]]

def get_zhihu_hot_search() -> List[SearchWord]:
    """
    获取知乎热搜数据
    """
    url = "https://www.zhihu.com/api/v3/feed/topstory/hot-lists/total?limit=50"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    try:
        response = requests.get(url, headers=headers)

        if not response.ok:
            print(f"请求失败: {response.status_code} - {response.reason}")
            return []

        result: TopSearch = response.json()
        # words = result.get('top_search', {}).get('words', [])

        return result

    except Exception as e:
        print(f"获取知乎热搜时出错: {e}")
        return []

In [14]:
data = get_zhihu_hot_search()

请求失败: 403 - Forbidden


In [21]:
import json
import logging
from bs4 import BeautifulSoup
from requests import Session

# 配置日志记录器
logger = logging.getLogger(__name__)

class HotSearch:
    def __init__(self):
        # 热搜URL（根据实际情况替换）
        self.HOT_SEARCH_URL2 = "https://www.zhihu.com/topsearch"  # 请替换为实际的URL

    def request_session(self):
        """创建并返回一个请求会话"""
        session = Session()
        # 可以在这里添加一些默认的请求头
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        return session

    def get_hot_search(self):
        items = []
        resp = None
        try:
            with self.request_session() as s:
                resp = s.get(self.HOT_SEARCH_URL2, timeout=10)
                resp.raise_for_status()  # 检查请求是否成功

                soup = BeautifulSoup(resp.text, 'html.parser')
                script = soup.find('script', type='text/json', id='js-initialData')

                if script:
                    obj = json.loads(script.string)
                    # 根据实际JSON结构调整路径
                    items = obj.get('initialState', {}).get('topsearch', {}).get('data', [])
                else:
                    logger.warning('未找到指定的script标签')

        except Exception as e:
            logger.exception('获取热搜失败')
            # 可以选择在这里返回空列表或者重新抛出异常

        return items

# 使用示例
if __name__ == "__main__":
    hot_search = HotSearch()
    items = hot_search.get_hot_search()

    if items:
        print(f"获取到 {len(items)} 条热搜:")
        for i, item in enumerate(items[:5], 1):  # 只显示前5条
            print(f"{i}. {item.get('queryDisplay', '无标题')} - {item.get('queryDescription', '无描述')}")
    else:
        print("未获取到热搜数据")

获取到 24 条热搜:
1. 为什么要实行薪酬保密 - 
2. 老师怎么看出来作文是抄的 - 
3. 为什么有些人的预判能力这么强 - 
4. 你的学校发生过最扯的事是什么 - 
5. 中国有多少城市能撑起来七日游 - 


In [22]:
items

[{'queryDisplay': '为什么要实行薪酬保密',
  'realQuery': '为什么要实行薪酬保密',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 48905,
  'isNew': False},
 {'queryDisplay': '老师怎么看出来作文是抄的',
  'realQuery': '老师怎么看出来作文是抄的',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 9646,
  'isNew': False},
 {'queryDisplay': '为什么有些人的预判能力这么强',
  'realQuery': '为什么有些人的预判能力这么强',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 19563,
  'isNew': False},
 {'queryDisplay': '你的学校发生过最扯的事是什么',
  'realQuery': '你的学校发生过最扯的事是什么',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 6132,
  'isNew': False},
 {'queryDisplay': '中国有多少城市能撑起来七日游',
  'realQuery': '中国有多少城市能撑起来七日游',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': -5,
  'isNew': False},
 {'queryDisplay': '你听过最野的野史有多野',
  'realQuery': '你听过最野的野史有多野',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': -6,
  'isNew': False},
 {'queryDisplay': 

In [23]:
url = "https://www.zhihu.com/topsearch"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Cookie': 'SUB=_2AkMVdRtlf8NxqwJRmfoWy2_lb4V0yQvEieKjKeq-JRMxHRl-yT8XqmYatRB6PvU1ijEk4CykabQQvFhJAy31x99v4Ejs;'
}


response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

if not response.ok:
    print(f"请求失败: {response.status_code} - {response.reason}")

soup = BeautifulSoup(response.text, 'html.parser')
script = soup.find('script', type='text/json', id='js-initialData')
obj = json.loads(script.string)


In [26]:
words = obj.get('initialState', {}).get('topsearch', {}).get('data', [])

In [28]:
words

[{'queryDisplay': '为什么要实行薪酬保密',
  'realQuery': '为什么要实行薪酬保密',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 48905,
  'isNew': False},
 {'queryDisplay': '老师怎么看出来作文是抄的',
  'realQuery': '老师怎么看出来作文是抄的',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 9646,
  'isNew': False},
 {'queryDisplay': '为什么有些人的预判能力这么强',
  'realQuery': '为什么有些人的预判能力这么强',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 19563,
  'isNew': False},
 {'queryDisplay': '你的学校发生过最扯的事是什么',
  'realQuery': '你的学校发生过最扯的事是什么',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': 6132,
  'isNew': False},
 {'queryDisplay': '中国有多少城市能撑起来七日游',
  'realQuery': '中国有多少城市能撑起来七日游',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': -5,
  'isNew': False},
 {'queryDisplay': '你听过最野的野史有多野',
  'realQuery': '你听过最野的野史有多野',
  'queryDescription': '',
  'redirectLink': '',
  'type': 0,
  'status': -6,
  'isNew': False},
 {'queryDisplay': 

In [32]:
import requests

url = "https://api.ba9.cn/api/get.douyinhot?type=douyin"
headers = {'Authorization': 'YOUR_TOKEN'}

response = requests.get(url, headers=headers)
data = response.json()
print(data)

{'success': True, 'title': '抖音', 'subtitle': '热搜榜', 'update_time': '2025-09-13 14:34:01', 'data': [{'index': 1, 'title': '第一视角看钱塘江大潮', 'hot': '1103.2万', 'label': 3, 'url': 'https://www.douyin.com/search/%E7%AC%AC%E4%B8%80%E8%A7%86%E8%A7%92%E7%9C%8B%E9%92%B1%E5%A1%98%E6%B1%9F%E5%A4%A7%E6%BD%AE', 'mobilUrl': 'https://www.douyin.com/search/%E7%AC%AC%E4%B8%80%E8%A7%86%E8%A7%92%E7%9C%8B%E9%92%B1%E5%A1%98%E6%B1%9F%E5%A4%A7%E6%BD%AE'}, {'index': 2, 'title': '沉浸式感受808米高空独木桥', 'hot': '1102万', 'label': 3, 'url': 'https://www.douyin.com/search/%E6%B2%89%E6%B5%B8%E5%BC%8F%E6%84%9F%E5%8F%97808%E7%B1%B3%E9%AB%98%E7%A9%BA%E7%8B%AC%E6%9C%A8%E6%A1%A5', 'mobilUrl': 'https://www.douyin.com/search/%E6%B2%89%E6%B5%B8%E5%BC%8F%E6%84%9F%E5%8F%97808%E7%B1%B3%E9%AB%98%E7%A9%BA%E7%8B%AC%E6%9C%A8%E6%A1%A5'}, {'index': 3, 'title': '家国守候 只待君归', 'hot': '1084.8万', 'label': 0, 'url': 'https://www.douyin.com/search/%E5%AE%B6%E5%9B%BD%E5%AE%88%E5%80%99+%E5%8F%AA%E5%BE%85%E5%90%9B%E5%BD%92', 'mobilUrl': 'https://www.dou

In [31]:
data['data']

[{'index': 1,
  'title': '小鹏主动召回 47490 辆 P7 + 汽车，免费更换转向机总成，转向助力失效的安全隐患有多大？',
  'hotTag': None,
  'hot': '388万',
  'url': 'https://www.zhihu.com/question/1949881317593408069',
  'mobilUrl': 'https://www.zhihu.com/question/1949881317593408069'},
 {'index': 2,
  'title': '美国第二架 B-21 「突袭者」轰炸机完成首飞，有哪些细节值得关注？',
  'hotTag': 'hot',
  'hot': '323万',
  'url': 'https://www.zhihu.com/question/1949892246443877786',
  'mobilUrl': 'https://www.zhihu.com/question/1949892246443877786'},
 {'index': 3,
  'title': '南孚的不可充电电池真的不可充电吗？',
  'hotTag': None,
  'hot': '239万',
  'url': 'https://www.zhihu.com/question/637467324',
  'mobilUrl': 'https://www.zhihu.com/question/637467324'},
 {'index': 4,
  'title': '蟑螂不能踩死，否则会走到哪里传播到哪里，是真的吗？',
  'hotTag': None,
  'hot': '198万',
  'url': 'https://www.zhihu.com/question/767911491',
  'mobilUrl': 'https://www.zhihu.com/question/767911491'},
 {'index': 5,
  'title': '布冯最近向国际足联提出，现代足球比赛的球门偏小，希望国际足联增大球门尺寸，这个你怎么看？',
  'hotTag': None,
  'hot': '167万',
  'url': 'https://www.z

In [34]:
from enum import Enum

class HotSearchType(str, Enum):
    """热搜平台类型枚举"""
    WEIBO = "微博"
    DOUYIN = "抖音"
    BAIDU = "百度"
    ZHIHU = "知乎"
    TOUTIAO = "今日头条"
    BILI = "哔哩哔哩"

    @classmethod
    def list(cls):
        return [t.value for t in cls]

HotSearchType.list()

['微博', '抖音', '百度', '知乎', '今日头条', '哔哩哔哩']